In [40]:
import numpy as np
import pandas as pd
import os
import sys

# Add the directory containing the VotingModel to the Python path
sys.path.append(os.path.abspath('/Users/idrees/Code/govxs/'))

from model.VotingModel import VotingModel

def simulate_bribery_mean(model, target_project, desired_increase):
    num_voters, num_projects = model.voting_matrix.shape
    new_voting_matrix = model.voting_matrix.copy()
    original_voting_matrix=new_voting_matrix

    # Calculate the original allocation
    original_allocation = model.mean_aggregation()
    original_funds = original_allocation[target_project]

    # Calculate the target allocation
    target_funds = original_funds + desired_increase
    total_required_votes = target_funds * num_voters

    # Calculate the current votes for the target project
    current_votes = np.sum(new_voting_matrix[:, target_project])
    votes_needed = total_required_votes - current_votes

    bribery_cost = 0
    current_increase = 0

    # Adjust the votes to achieve the desired increase
    for voter in range(num_voters):
        if current_increase >= votes_needed:
            break
        available_funds = new_voting_matrix[voter].sum() - new_voting_matrix[voter, target_project]
        if available_funds > 0:
            shift_amount = min(available_funds, votes_needed - current_increase)
            new_voting_matrix[voter, target_project] += shift_amount
            bribery_cost += shift_amount
            current_increase += shift_amount

    # Calculate the new allocation after bribery
    model.voting_matrix = new_voting_matrix
    new_allocation = model.mean_aggregation()
    
    print("Mean Aggregation:")
    print("Original Voting Matrix:\n", original_voting_matrix)
    print("New Voting Matrix:\n", new_voting_matrix)
    print("Original Allocation:\n", original_allocation)
    print("New Allocation:\n", new_allocation)
    
    return bribery_cost, new_voting_matrix, new_allocation

def simulate_bribery_quadratic(model, target_project, desired_increase):
    num_voters, num_projects = model.voting_matrix.shape
    new_voting_matrix = model.voting_matrix.copy()
    original_voting_matrix=new_voting_matrix

    # Calculate the original allocation
    original_allocation = model.quadratic_aggregation()
    original_funds = original_allocation[target_project]

    # Calculate the target allocation
    target_funds = original_funds + desired_increase
    total_required_votes = (target_funds * np.sum(original_allocation)) ** 0.5

    # Calculate the current votes for the target project
    current_votes = np.sum(new_voting_matrix[:, target_project] ** 2)
    votes_needed = total_required_votes - np.sqrt(current_votes)
    votes_needed = votes_needed ** 2

    bribery_cost = 0
    current_increase = 0

    # Adjust the votes to achieve the desired increase
    while current_increase < votes_needed:
        for voter in range(num_voters):
            if current_increase >= votes_needed:
                break
            available_funds = (new_voting_matrix[voter].sum() ** 2) - (new_voting_matrix[voter, target_project] ** 2)
            if available_funds > 0:
                shift_amount = min(available_funds, votes_needed - current_increase)
                new_voting_matrix[voter, target_project] += np.sqrt(shift_amount)
                bribery_cost += shift_amount
                current_increase += shift_amount

    # Calculate the new allocation after bribery
    model.voting_matrix = new_voting_matrix
    new_allocation = model.quadratic_aggregation()

    print("Median Aggregation:")
    print("Original Voting Matrix:\n", original_voting_matrix)
    print("New Voting Matrix:\n", new_voting_matrix)
    print("Original Allocation:\n", original_allocation)
    print("New Allocation:\n", new_allocation)
    
    return bribery_cost, new_voting_matrix, new_allocation

def simulate_bribery_median(model, target_project, desired_increase):
    num_voters, num_projects = model.voting_matrix.shape
    new_voting_matrix = model.voting_matrix.copy()
    original_voting_matrix=new_voting_matrix

    # Calculate the original allocation
    original_allocation = model.median_aggregation()
    original_funds = original_allocation[target_project]

    # Calculate the target allocation
    target_funds = original_funds + desired_increase

    bribery_cost = 0

    # Adjust the votes to achieve the desired increase
    votes = new_voting_matrix[:, target_project]
    median_vote = np.median(votes)
    while median_vote < target_funds:
        min_index = np.argmin(votes)
        max_vote = np.max(votes)
        shift_amount = target_funds - median_vote
        new_voting_matrix[min_index, target_project] += shift_amount
        bribery_cost += shift_amount
        votes = new_voting_matrix[:, target_project]
        median_vote = np.median(votes)
        if np.max(votes) > max_vote:
            break

    # Calculate the new allocation after bribery
    model.voting_matrix = new_voting_matrix
    new_allocation = model.median_aggregation()
    print("Mean Aggregation:")
    print("Original Voting Matrix:\n", original_voting_matrix)
    print("New Voting Matrix:\n", new_voting_matrix)
    print("Original Allocation:\n", original_allocation)
    print("New Allocation:\n", new_allocation)
    
    
    return bribery_cost, new_voting_matrix, new_allocation

def evaluate_bribery_impact(model, target_project, desired_increase):
    # Original allocations without bribery
    original_allocations = {}
    for method in ["mean", "median", "quadratic"]:
        original_allocations[method] = model.allocate_funds(method)[target_project]

    # Bribery impact for each aggregation method
    bribery_results = []
    for method in ["mean", "median", "quadratic"]:
        if method == "mean":
            bribery_cost, new_voting_matrix, new_allocation= simulate_bribery_mean(
                model, target_project, desired_increase
            )
        elif method == "median":
            bribery_cost, new_median_voting_matrix, new_allocation = simulate_bribery_median(
                model, target_project, desired_increase
            )
        elif method == "quadratic":
            bribery_cost, new_voting_matrix, new_allocation = simulate_bribery_quadratic(
                model, target_project, desired_increase
            )
        increase = new_allocation[target_project] - original_allocations[method]
        bribery_results.append({
            "method": method,
            "bribery_cost": bribery_cost,
            "original_fund_allocation":original_allocations[method],
            "new_fund_allocations": new_allocation[target_project],
            "increase": increase
        })

    return pd.DataFrame(bribery_results)
# Initialize the model
model = VotingModel(num_voters=4, num_projects=3, total_op_tokens=30e6)
model.step()

# Set parameters for bribery evaluation
target_project = 1
desired_increase = 10e6  # Desired increase of 5M OP tokens

# Evaluate the impact of bribery
bribery_impact_df = evaluate_bribery_impact(model, target_project, desired_increase)
bribery_impact_df.head()



Mean Aggregation:
Original Voting Matrix:
 [[ 8643741.03654362 30000000.          4119765.56523525]
 [21835547.47045131 30000000.          5095424.3698714 ]
 [ 5268487.97591018 12739727.81279312 12297305.7691951 ]
 [18085673.07848217   843312.23916039 11071014.68235745]]
New Voting Matrix:
 [[ 8643741.03654362 30000000.          4119765.56523525]
 [21835547.47045131 30000000.          5095424.3698714 ]
 [ 5268487.97591018 12739727.81279312 12297305.7691951 ]
 [18085673.07848217   843312.23916039 11071014.68235745]]
Original Allocation:
 [13458362.39034682  8395760.01298838  8145877.5966648 ]
New Allocation:
 [10093771.79276011 13796820.00974128  6109408.1974986 ]
Mean Aggregation:
Original Voting Matrix:
 [[ 8643741.03654362 30000000.          4119765.56523525]
 [21835547.47045131 30000000.          5095424.3698714 ]
 [ 5268487.97591018 19945305.90462233 12297305.7691951 ]
 [18085673.07848217 19945305.90462232 11071014.68235745]]
New Voting Matrix:
 [[ 8643741.03654362 30000000.       

/Users/idrees/Code/govxs/agents/VoterAgent.py:8: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


,method,bribery_cost,original_fund_allocation,new_fund_allocations,increase
0,mean,4.000000e+07,8.395760e+06,1.379682e+07,5.401060e+06
1,median,2.630757e+07,7.964115e+06,1.613895e+07,8.174839e+06
2,quadratic,4.087546e+14,6.650479e+06,2.353891e+07,1.688844e+07
